Nombre: Víctor

Apellidos: Toscano Durán

## Coche Montaña

En el problema del coche de montaña un vehículo debe conducir hacia la derecha para salir de un valle. Las paredes del valle son lo suficientemente empinadas como para que acelerar a ciegas hacia la meta con una velocidad insuficiente haga que el vehículo se detenga y se deslice hacia abajo. El agente debe aprender a acelerar primero hacia la izquierda para ganar suficiente impulso en el retorno y poder subir la colina.

El estado es la posición horizontal del vehículo x∈[−1.2,0.6] y la velocidad v∈[−0.07,0.07]. En cualquier paso de tiempo, el vehículo puede acelerar a la izquierda (a=−1), acelerar a la derecha (a=1), o dejarse llevar por la inercia (a=0). Recibimos una recompensa de −1 en cada cambio de sentido, y terminamos cuando el vehículo llega al lado derecho del valle más allá de x=0.6
.
Las transiciones en el problema del coche de montaña son deterministas:

- v′ = v+0.001 a−0.0025cos(3x)                                                                                                 -- x' = x+v′

El término gravitacional en la actualización de la velocidad es lo que impulsa al vehículo con poca potencia hacia el fondo del valle. Las transiciones se sujetan a los límites del espacio de estados.

El problema del coche de montaña es un buen ejemplo de un problema con retorno retardado. Se requieren muchas acciones para llegar al estado objetivo, lo que dificulta que un agente no entrenado reciba algo más que penalizaciones unitarias de forma continua. Los mejores algoritmos de aprendizaje son capaces de propagar eficazmente el conocimiento de las trayectorias que llegan a la meta al resto del espacio de estados.                                                                                 Implementar esto en Python y obtener la politica optima

In [1]:
import numpy as np

# Definir los límites del espacio de estados
POSICION_MINIMA = -1.2
POSICION_MAXIMA = 0.6
VELOCIDAD_MINIMA = -0.07
VELOCIDAD_MAXIMA = 0.07

# Definir los hiperparámetros del algoritmo de Q-learning
TASA_APRENDIZAJE = 0.1
FACTO_DESCUENTO = 0.99
EPSILON = 0.1
NUM_EPISODIOS = 10000

# Definir el número de divisiones para discretizar el espacio de estados
NUM_POSICIONES = 30
NUM_VELOCIDADES = 30
NUM_ACCIONES = 3

# Inicializar la tabla Q con valores aleatorios
Q = np.random.uniform(low=-1, high=1, size=(NUM_POSICIONES, NUM_VELOCIDADES, NUM_ACCIONES))

# Definir la función de discretización del espacio de estados
def discretizar_estado(posicion, velocidad):
    gap_posicion = (POSICION_MAXIMA - POSICION_MINIMA) / NUM_POSICIONES
    gap_velocidad = (VELOCIDAD_MAXIMA - VELOCIDAD_MINIMA) / NUM_VELOCIDADES
    
    indice_posicion = int((posicion - POSICION_MINIMA) / gap_posicion)
    indice_velocidad = int((velocidad - VELOCIDAD_MINIMA) / gap_velocidad)
    
    return indice_posicion, indice_velocidad

# Definir la función de selección de acción
def seleccionar_accion(estado):
    if np.random.uniform() < EPSILON:
        return np.random.randint(NUM_ACCIONES)
    else:
        return np.argmax(Q[estado[0], estado[1]])

# Bucle principal de entrenamiento
for episodio in range(NUM_EPISODIOS):
    # Inicializar el estado del episodio
    posicion = np.random.uniform(low=-0.6, high=-0.4)
    velocidad = 0.0
    estado = discretizar_estado(posicion, velocidad)
    
    # Bucle interno del episodio
    while True:
        # Seleccionar una acción
        accion = seleccionar_accion(estado)
        
        # Tomar la acción y obtener la siguiente posición y velocidad
        aceleracion = [-1.0, 0.0, 1.0][accion]
        velocidad += 0.001 * aceleracion - 0.0025 * np.cos(3 * posicion)
        velocidad = np.clip(velocidad, VELOCIDAD_MINIMA, VELOCIDAD_MAXIMA)
        posicion += velocidad
        posicion = np.clip(posicion, POSICION_MINIMA, POSICION_MAXIMA)
        
        # Obtener el nuevo estado discretizado
        siguiente_estado = discretizar_estado(posicion, velocidad)
        
        # Obtener la recompensa
        recompensa = -1 if posicion < 0.5 else 0
        
        # Actualizar la tabla Q
        Q[estado][accion] += TASA_APRENDIZAJE * (recompensa + FACTO_DESCUENTO * np.max(Q[siguiente_estado]) - Q[estado][accion])
        
        # Actualizar el estado actual
        estado = siguiente_estado
        
        # Terminar el episodio si se alcanza el estado objetivo
        if posicion >= 0.5:
            break

# Evaluar la política óptima
num_eval_episodios = 10
recompensas_totales = 0

for _ in range(num_eval_episodios):
    posicion = np.random.uniform(low=-0.6, high=-0.4)
    velocidad = 0.0
    
    while True:
        estado = discretizar_estado(posicion, velocidad)
        accion = np.argmax(Q[estado])
        
        aceleracion = [-1.0, 0.0, 1.0][accion]
        velocidad += 0.001 * aceleracion - 0.0025 * np.cos(3 * posicion)
        velocidad = np.clip(velocidad, VELOCIDAD_MINIMA, VELOCIDAD_MAXIMA)
        posicion += velocidad
        posicion = np.clip(posicion, POSICION_MINIMA, POSICION_MAXIMA)
        
        if posicion >= 0.5:
            recompensas_totales += 1
            break

recompensa_promedio = recompensas_totales / num_eval_episodios
print(f"La política óptima alcanza la meta en promedio {recompensa_promedio} veces en {num_eval_episodios} episodios.")

C:\Users\VictorToscano\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\VictorToscano\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\VictorToscano\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


La política óptima alcanza la meta en promedio 1.0 veces en 10 episodios.


### Explicación código

El codigo implementa el algoritmo de Q-learning para resolver el problem del coche de montaña. El objetivo es que el vehiculo aprenda a salir del valle acelerando primero hacia la izquierda para obtener suficiente impulso y luego subir la colina acelerando hacia la derecha.

El bucle principal de entrenamiento ejecuta un numero determindo de episodios. En cada episodio, se inicializa el estado del vehiculo de manera aleatoria en terminos de posicion y velocidad. A continuacion, se ejecuta un bucle interno que continua hasta que el vehiculo alcanza la posicion objetivo.

Dentro del bucle interno, se selecciona una accion basada en el estado actual del vehiculo. Se utiliza epsilon para equilibrar la exploracion y la explotacion. Si un numero aleatorio es menor que epsilon, se elige una accion al azar; de lo contrario, se selecciona la accion con el mayor valor de Q para ese estado.

Una vez que se selecciona la accion, se actualiza la posicion y velocidad del vehiculo de acuerdo con las ecuaciones de transicion del problema del coche de montaña. Luego, se calcula la recompensa para el estado actual y se actualiza la tabla Q utilizando el algoritmo de Q-learning. La tabla Q almacena los valores de Q para cada estado y accion, y se actualiza en funcion de las recompensas obtenidas y los valores de Q para los estados siguientes.

Despues de finalizar el bucle de entrenamiento, se evalua la politica optima obtenida. Esto implica ejecutar un numero de episodios de evaluacion y calcular la cantidad promedio de veces que el vehiculo alcanza la posicion objetivo.

Finalmente, se muestra en pantalla el promedio de las veces que el vehiculo alcanza la posicion objetivo durante la evaluacion(similar, al de entrenamiento, pero seleccionando la mejor accion segun los valores de Q), lo que indica la efectividad de la politica optima aprendida.